In [ ]:
import numpy as np
import pygame

pygame 2.4.0 (SDL 2.26.4, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Code de la dynamique de la simulation

In [2]:
def count(matrix, idx:tuple):
    """
    Matrix: La matrice à la génération voulue
    Idx: Coordonnées (i,j) de la cellule étudiée
    On compte le nombre de cellules voisines vivantes de la cellule étudiée
    """
    i, j = idx
    v = 0 # initialisation du nombre de cellules voisines vivantes
    for k in [-1, 0, 1]:
        for l in [-1, 0, 1]:
            if k != 0 or l != 0:
                if matrix[np.mod(i+k, len(matrix)),np.mod(j+l, len(matrix[0]))] == 1: # on intruduit un modulo de manière à ne pas dépasser les bords dans le décompte des indices. On aura donc un monde dont les extrêmités se rejoignent.
                    v += 1
    return v
                
        

In [3]:
def life(matrix, idx:tuple):
    """
    Matrix: La matrice à la génération voulue
    Idx: Coordonnées (i,j) de la cellule étudiée
    On décide de la survie / naissance / mort d'une cellule
    """
    i, j = idx
    state = matrix[i,j]
    v = count(matrix, idx)
    if v == 2:
        return state
    if v == 3:
        return 1
    else:
        return 0

In [4]:
def next_matrix(matrix):
    """
    On crée une nouvelle matrice correspondant à l'instant d'après celle qu'on lui donne
    """
    next_matrix = np.zeros((len(matrix), len(matrix[0])))
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            next_matrix[i,j] = life(matrix, (i,j))
    return next_matrix

In [5]:
def game_all_frames(matrix, nb_gen): # ça c'est pas nécessaire
    """
    On enregistre les différents états de la matrice au fil des générations
    """
    all_frames = np.zeros((nb_gen+1, len(matrix) ,len(matrix[0])))
    all_frames[0] = matrix
    last_matrix = matrix
    for gen in range(nb_gen):
        all_frames[gen+1] = next_matrix(all_frames[gen])
    return all_frames

In [6]:
matrice2 = np.random.randint(0, 2, (155, 80))

In [7]:
def glider():
    matrice2[0,0] = 1
    matrice2[0,1] = 1
    matrice2[1,1] = 1
    matrice2[1,2] = 1
    matrice2[2,0] = 1

# Affichage de la simulation

In [ ]:
pygame.init() # jsp trop pk mais ok
start_matrix = matrice2.copy()
matrix = start_matrix.copy()
pixel_size = 10
max_scale = (pixel_size*len(matrix), pixel_size*len(matrix[0]))
win = pygame.display.set_mode(max_scale) # définition de la taille de win (la fenêtre de jeu)
pygame.display.set_caption("Game python") # nom du jeu
run = True
time_delay = 5
pause = False
p_pressed = False
mouse_pressed = False
space_pressed = False
r_pressed = False

def drawGame():
      win.fill((0, 0, 0))
      for i in range(len(matrix)):
            for j in range(len(matrix[0])):
                  if matrix[i,j] == 1:
                        pygame.draw.rect(win, (255, 255, 255), (i*pixel_size, j*pixel_size, pixel_size, pixel_size))
      # ici il faudra mettre l'état de la matrice
      pygame.display.update()


            

while run:
      pygame.time.delay(time_delay)
      keys = pygame.key.get_pressed()
      
      for event in pygame.event.get():
            if event.type == pygame.QUIT:
                  run = False
      
      
      if keys[pygame.K_p]:
            p_pressed = True
      if p_pressed:
            if not keys[pygame.K_p]:
                  p_pressed = False
                  pause = not pause # on inverse la valeur de pause, cela évite les conditions de vérification pour savoir dans quel état mettre pause
      
      if keys[pygame.K_SPACE]:
            space_pressed = True
      if space_pressed:
            if not keys[pygame.K_SPACE]:
                  space_pressed = False
                  matrix = np.zeros((len(matrix),len(matrix[0])))

      if keys[pygame.K_r]:
            r_pressed = True
      if r_pressed:
            if not keys[pygame.K_r]:
                  r_pressed = False
                  matrix = start_matrix
      
      if pygame.mouse.get_pressed()[0]:
            mouse_pressed = True
      
      if mouse_pressed and not pygame.mouse.get_pressed()[0]:
            mouse_pressed = False
            n, m = pygame.mouse.get_pos()
            n //= pixel_size
            m//= pixel_size
            if matrix[n,m] == 0:
                  matrix[n,m] = 1
            else:
                  matrix[n,m] = 0
            drawGame()
                  
            
      if not pause:
            matrix = next_matrix(matrix)
            drawGame()
          
pygame.quit()   

#### Commandes:  
- Cliquer sur une cellule pour changer son état (vivante ou morte)
- "p" pour mettre sur pause
- "r" pour redémarrer la simulation avec la matrice d'origine
- touche espace pour effacer toutes les cellules

# Autre méthode

In [ ]:
def next_dic(dic, shape):
    trans_dic = dict()
    final_dic = dict()
    for elt in dic:
        i,j = elt
        for idx in [(-1,-1), (-1,0), (-1,1), (0,-1), (0,1), (1,-1), (1,0), (1,1)]:
            k,l = idx
            trans_dic[(np.mod(i+k, shape[0]), np.mod(j+l, shape[1]))] = trans_dic.get((np.mod(i+k, shape[0]), np.mod(j+l, shape[1])), 0) + 1 # on va chercher l'élément (i+k, j+l)
    for elt in trans_dic.items():
        key, value = elt
        if value == 2:
            try:
                final_dic[key] = dic[key] # si la cellule a deux voisins, elle ne change pas d'état
            except KeyError:
                pass
        elif value == 3: # si la cellule a trois voisins, elle reste / devient vivante
            final_dic[key] = 1
            # par souci d'optimisation, on ne tient pas compte des cellules mortes donc pas besoin de regarder les autres cas
    return final_dic
            

In [ ]:
def matrix_as_dic(matrix):
    dic = dict()
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            if matrix[i,j] == 1:
                dic[i,j] = 1
    return dic
    

In [ ]:
matrice2 = np.random.randint(0, 2, (50, 50))

# Affichage selon la seconde méthode

In [ ]:
pygame.init() # jsp trop pk mais ok
start_matrix = matrice2.copy()
matrix = start_matrix.copy()
dic = matrix_as_dic(matrix)
pixel_size = 10
max_scale = (pixel_size*len(matrix), pixel_size*len(matrix[0]))
win = pygame.display.set_mode(max_scale) # définition de la taille de win (la fenêtre de jeu)
pygame.display.set_caption("Game python") # nom du jeu
run = True
time_delay = 5
pause = False
p_pressed = False
mouse_pressed = False
space_pressed = False
r_pressed = False


def drawGame():
      win.fill((0, 0, 0))
      for coordinates in dic:
            i,j = coordinates
            pygame.draw.rect(win, (255, 255, 255), (i*pixel_size, j*pixel_size, pixel_size, pixel_size))
      # ici il faudra mettre l'état de la matrice
      pygame.display.update()


            
while run:
      pygame.time.delay(time_delay)
      keys = pygame.key.get_pressed()
      
      for event in pygame.event.get():
            if event.type == pygame.QUIT:
                  run = False
      
      
      if keys[pygame.K_p]:
            p_pressed = True
      if p_pressed:
            if not keys[pygame.K_p]:
                  p_pressed = False
                  pause = not pause # on inverse la valeur de pause, cela évite les conditions de vérification pour savoir dans quel état mettre pause
      
      if keys[pygame.K_SPACE]:
            space_pressed = True
      if space_pressed:
            if not keys[pygame.K_SPACE]:
                  space_pressed = False
                  dic = dict()
                  drawGame()

      if keys[pygame.K_r]:
            r_pressed = True
      if r_pressed:
            if not keys[pygame.K_r]:
                  r_pressed = False
                  dic = matrix_as_dic(start_matrix)
                  drawGame()
      
      if pygame.mouse.get_pressed()[0]:
            mouse_pressed = True
      
      if mouse_pressed and not pygame.mouse.get_pressed()[0]:
            mouse_pressed = False
            n, m = pygame.mouse.get_pos()
            n //= pixel_size
            m//= pixel_size
            if (n,m) not in dic:
                  dic[(n,m)] = 1
            else:
                  dic.pop((n,m))
            drawGame()
                  
            
      if not pause:
            dic = next_dic(dic, (len(matrix), len(matrix[0])))
            drawGame()
          
pygame.quit()   